In [3]:
pts = []
for line in ps.splitlines():
    if line.startswith('[['):
        pts.append(line)

In [7]:
pt2 = []
for l in pts:
    x = l.replace('[[', '')
    x = x.replace(']]', '')
    pt = [float(i) for i in x.split(' ') if i]
    pt2.append(pt)

In [8]:
len(pt2)

1000

In [2]:
import pickle

In [10]:
with open('pts-1000-co2-ivr', 'wb') as f:
    pickle.dump(pt2, f)

In [3]:
pt2_ivr_co2 = pickle.load(open('pts-1000-co2-ivr', 'rb'))

In [4]:
len(pt2_ivr_co2)

1000

In [5]:
%cd ../

/Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu


In [6]:
import numpy as np
import pickle
import emukit as ek
import GPy

from emukit.model_wrappers import GPyModelWrapper
from emukit.experimental_design.experimental_design_loop import ExperimentalDesignLoop
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.core.initial_designs import RandomDesign, latin_design
from emukit.experimental_design.acquisitions import ModelVariance, IntegratedVarianceReduction
from emukit.core.loop import UserFunctionWrapper
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop
from emukit.core.optimization import GradientAcquisitionOptimizer

from sumo_grid_simulation.grid_simulation import Simulator
import experimental_design.config as config

In [7]:
def user_function_co2(X):
    """  X = inputs - emukit doesnt pass named args, just an NxM ndarray, N is the number of points to evaluate, M is the number of parameters per each point """
    result = []
    i = 0
    
    print(X)
    print(f'\nUser function time loss called with {X.shape[0]} inputs to simulate')

    
    for gridSize, edgeMaxSpeed, maxSpeed, edgeLength, numLanes, accel in X:
        print(f'\nEvaluating input: {i+1} of {X.shape[0]}\n')
        
        alpha = 0.005
        max_number_of_vehicles = ((gridSize - 1) * gridSize * 2 + 4 * gridSize) * edgeLength / 5
        period = 300/(max_number_of_vehicles * alpha)
        
        s = simulator.simulate(
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            maxSpeed      = maxSpeed,
            edgeLength    = edgeLength,
            numberOfLanes = int(numLanes),
            accel         = accel,
            trips_generator_period = period
        )
        # average co2 / average route duration
        CO2 = s['CO2']/s['num_emissions_samples']
        # average emissions / average route length
        result.append(CO2/s['duration'])
        i += 1
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

In [8]:
parameter_space = config.get_parameter_space()

In [10]:
simulator = Simulator(end_time=300)

In [11]:
design = RandomDesign(parameter_space)
test_X = design.get_samples(100)
test_Y = user_function_co2(test_X)

[[20.         18.23578545 23.77441153 35.66609777  1.          3.64992579]
 [17.         16.01206489 13.19969521 41.73154209  2.          2.71103827]
 [14.         12.14987726 10.18847345 50.10793591  2.          2.72478985]
 [11.          9.27804431 22.69320499 30.86801275  3.          1.76705981]
 [ 8.         12.75564786 12.84812472 39.0198911   1.          4.12725705]
 [14.          9.38346211  9.51239238 64.5938367   1.          4.55156648]
 [14.         13.73289943  7.62874785 45.30564723  2.          3.08919839]
 [14.         14.03939368 21.97411553 41.38399273  3.          2.02787133]
 [19.          8.76285164 18.18623474 64.56155311  1.          3.49758754]
 [12.         20.20362311 10.12357386 35.92082775  2.          3.28593778]
 [20.         20.05562584 25.78332819 65.25759837  1.          1.50899572]
 [ 4.         18.07086641 10.89476867 63.75695994  2.          1.97664227]
 [ 5.         17.50640333 26.99802991 68.95550697  2.          2.87119545]
 [ 8.         22.16128026

 Retrying in 1 seconds

Evaluating input: 14 of 100

 Retrying in 1 seconds

Evaluating input: 15 of 100

 Retrying in 1 seconds

Evaluating input: 16 of 100

 Retrying in 1 seconds

Evaluating input: 17 of 100

 Retrying in 1 seconds

Evaluating input: 18 of 100

 Retrying in 1 seconds

Evaluating input: 19 of 100

 Retrying in 1 seconds

Evaluating input: 20 of 100

 Retrying in 1 seconds

Evaluating input: 21 of 100

 Retrying in 1 seconds

Evaluating input: 22 of 100

 Retrying in 1 seconds

Evaluating input: 23 of 100

 Retrying in 1 seconds

Evaluating input: 24 of 100

 Retrying in 1 seconds

Evaluating input: 25 of 100

 Retrying in 1 seconds

Evaluating input: 26 of 100

 Retrying in 1 seconds

Evaluating input: 27 of 100

 Retrying in 1 seconds

Evaluating input: 28 of 100

 Retrying in 1 seconds

Evaluating input: 29 of 100

 Retrying in 1 seconds

Evaluating input: 30 of 100

 Retrying in 1 seconds

Evaluating input: 31 of 100

 Retrying in 1 seconds

Evaluating input: 32 o

In [ ]:
emulator_rnd = GPy.models.GPRegression(train_X, train_Y)
emukit_model_rnd = GPyModelWrapper(emulator_rnd, n_restarts=5)
emukit_model_rnd.optimize()
emulator_rnd

In [ ]:
for j in range(0, 1000, 50):
    pts = pt2_ivr_co2
    emulator = GPy.models.GPRegression(train_X, train_Y)
    emukit_model_rnd = GPyModelWrapper(emulator_rnd, n_restarts=5)
    emukit_model_rnd.optimize()
    emulator_rnd